# Natural Language Processing Exercise 2

One hot encoding

Using same sms spam dataset: https://www.kaggle.com/uciml/sms-spam-collection-dataset

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('../data/spam.csv', encoding = "ISO-8859-1")
data = data.iloc[:, :2]
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
from nltk.tokenize import wordpunct_tokenize
corpus = data['v2']
def tokenize(string):
    # lowercase
    lowered = string.lower()
    
    tokens = wordpunct_tokenize(lowered)
    return tokens
tokenized = [tokenize(message) for message in corpus]
print(tokenized[0])

In [6]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# TODO: We'll need to figure out a way to include the wordnet corpus with the kernel
# without downloading it in the kernel itself
nltk.download('wordnet')

In [9]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

In [41]:
def lemmatize_stem(word):
    word = lemmatizer.lemmatize(word, pos='v')
    word = stemmer.stem(word)
    return word

processed = [[lemmatize_stem(word) for word in message] for message in tokenized]